In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    data['DOLocationID'] = data['DOLocationID'].astype(str)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Load Raw Data

In [10]:
hub = 'Lga'
tune_hyp_params = False

In [11]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [12]:
rawdata = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [13]:
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,1,2018-01-01,0,0.0
1,2,2018-01-01,0,0.0


In [14]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
0,2018-01-01,0,0,0,0,0,0,2,1,0,...,0,0,1,0,1,0,0,1,0,0
1,2018-01-01,1,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2018-01-01,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Burrough Level Aggregation

In [15]:
zones = pd.read_csv('/home/urwa/Documents/Projects/NYU Remote/project/UrbanTemporalNetworks/Data/taxi_zones.csv')
zones['LocationID'] = zones['LocationID'].astype(str)
zones.head(2)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens


In [16]:
zontoBorough = dict(zip(zones.LocationID.values,zones.borough.values))

In [17]:
zone_weights = get_weights(rawdata, zontoBorough)
zone_weights.head(2)

,Borough,DOLocationID,zone_weight
0,Bronx,119,0.024243
1,Bronx,126,0.017055


In [18]:
rawdata['DOLocationID'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count,Borough
0,EWR,2018-01-01,0,0.0,EWR
1,Queens,2018-01-01,0,0.0,Queens


In [19]:
borough_data = getTimeSeries(rawdata)
borough_data = borough_data.reset_index()
borough_data.head(3)

DOLocationID,Date,Hour,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
0,2018-01-01,0,8,40,0,55,36,3
1,2018-01-01,1,1,2,0,11,9,0
2,2018-01-01,2,1,0,0,0,1,0


### Merge External Data Features

In [20]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [21]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,18/1/1 1:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [22]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [23]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [24]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [25]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [26]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [27]:
extDf = extDf[selected_columns]

In [28]:
print(borough_data.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [29]:
borough_data['Date'] = pd.to_datetime(borough_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [30]:
borough_data = pd.merge(borough_data,extDf, on=['Date', 'Hour'], how='inner')
print(borough_data.shape)
borough_data['Date'] = borough_data['Date'].dt.date
borough_data.head()

(8760, 20)


,Date,Hour,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island,Dow,arrival,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,8,40,0,55,36,3,0,3,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01,1,1,2,0,11,9,0,0,0,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
2,2018-01-01,2,1,0,0,0,1,0,0,1,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
3,2018-01-01,3,0,1,0,0,0,0,0,0,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
4,2018-01-01,4,0,0,1,3,9,0,0,0,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [31]:
borough_data.columns

Index(['Date', 'Hour', 'Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens',
       'Staten Island', 'Dow', 'arrival', 'maxtemp', 'mintemp', 'avgtemp',
       'departure', 'hdd', 'cdd', 'participation', 'newsnow', 'snowdepth',
       'ifSnow'],
      dtype='object')

In [32]:
lagColumns = ['Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens',
       'Staten Island', 'arrival']

DateColumns = ['Date']

targetColumns = ['Bronx', 'Brooklyn', 'EWR', 'Manhattan', 'Queens',
       'Staten Island']

In [33]:
maxlag = 12

borough_data_lag = addLag(borough_data, maxlag, lagColumns)

borough_data_lag.shape

(8748, 104)

In [34]:
CommR2List = []
EdgeR2List = []
residualDf_list = []

for m in range(1,13):
    print()
    print("month: ",m)
    month_index  = pd.to_datetime(borough_data_lag.Date).dt.month == m

    dataset_train = borough_data_lag[~month_index]
    dataset_test = borough_data_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)

    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_testData = edge_data[edgeMonthIndex]
    select_cols = [c for c in edge_testData.columns if c not in ['Date','Hour']]
    edge_testData = edge_testData[select_cols]
    print("edge test data shape: ",edge_testData.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]

    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    comm_prediction = rf2.predict(X_test)
    edge_prediction_df = pd.DataFrame(comm_prediction)
    edge_prediction_df.columns = y_test.columns

    residual = y_test - comm_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    boroughs = list(edge_prediction_df.columns)
    for bor in boroughs:
    #     print(bor)

        weight_df = zone_weights[zone_weights.Borough == bor]

    #     print(len(weight_df.DOLocationID))

        for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
            edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight


    select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
    edge_prediction_df = edge_prediction_df[select_cols]


    edge_prediction_df = edge_prediction_df[edge_testData.columns]

    edge_r2 = r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)

    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)


month:  1
Train Size:  (8016, 104)
Test Size:  (732, 104)
edge test data shape:  (732, 257)
Train R2:  0.997803230047267
Test R2:  0.9175850070448057
Edge R2:  0.5906519140085869

month:  2
Train Size:  (8076, 104)
Test Size:  (672, 104)
edge test data shape:  (672, 257)
Train R2:  0.9977780197806484
Test R2:  0.9298280844799991
Edge R2:  0.6285245411098802

month:  3
Train Size:  (8004, 104)
Test Size:  (744, 104)
edge test data shape:  (744, 257)
Train R2:  0.9977706554916567
Test R2:  0.9395411229919556
Edge R2:  0.6480613559482848

month:  4
Train Size:  (8028, 104)
Test Size:  (720, 104)
edge test data shape:  (720, 257)
Train R2:  0.9977846597987112
Test R2:  0.9442848078771623
Edge R2:  0.6328175751902221

month:  5
Train Size:  (8004, 104)
Test Size:  (744, 104)
edge test data shape:  (744, 257)
Train R2:  0.9977519906124589
Test R2:  0.9381767166263386
Edge R2:  0.6369488264998405

month:  6
Train Size:  (8028, 104)
Test Size:  (720, 104)
edge test data shape:  (720, 257)
Tra

In [35]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.929182663849621
0.6229531206712269


In [36]:
res_df = pd.concat(residualDf_list, axis = 0)
print(res_df.shape)
res_df.head()

(8748, 8)


,Date,Hour,Bronx,Brooklyn,EWR,Manhattan,Queens,Staten Island
12,2018-01-01,12,4.428889,15.193333,1.790000,-23.426667,-19.216667,-0.533333
13,2018-01-01,13,-5.185556,14.632222,-0.160000,-8.141111,-8.582222,-0.471111
14,2018-01-01,14,-1.558889,13.484444,-0.512222,54.581111,3.124444,-0.643333
15,2018-01-01,15,1.080000,23.395556,-1.403333,55.465556,4.008889,-0.605556
16,2018-01-01,16,-5.245556,17.460000,1.480000,-24.355556,-31.418889,-0.684444


In [37]:
res_df.to_csv('/home/urwa/Documents/Projects/NYU Remote/project/data/residuals/lga_borough.csv')